In [2]:
# Instalar bibliotecas
# Instalar Librosa (essencial para áudio)

!pip install librosa
!pip install xxd
!pip install numpy
!pip install sklearn
!pip install zipfile
!pip install os
!pip install matplotlib
!pip install pandas
!pip install seaborn
!pip install keras
!pip install pydub

ERROR: Could not find a version that satisfies the requirement xxd (from versions: none)
ERROR: No matching distribution found for xxd
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
ERROR: Could not find a version that satisfies the requirement zipfile (from versions: none)
ERROR: No matching distribution found for zipfile
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [1]:
# Instalando TensorFlow

!pip install tensorflow==2.16.1

In [1]:
# Upgrade ml_dtypes to a compatible version
!pip install --upgrade ml_dtypes

In [2]:
# Imports e Carregamento de Dados
import os
import zipfile
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import librosa
from sklearn.model_selection import train_test_split

print("--- Descompactando o arquivo data.zip ---")
# Certifique-se de que seu data.zip foi carregado no Colab
with zipfile.ZipFile('data.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')
print("--- Arquivos descompactados. ---")

DATASET_PATH = 'dataset/data'
KEYWORDS = ['ligar', 'desligar', 'noise'] # Verifique se corresponde às suas pastas

--- Descompactando o arquivo data.zip ---
--- Arquivos descompactados. ---


In [3]:
# Pré-processamento (Espectrogramas)
print("\n--- Processando áudios para espectrogramas ---")
def get_spectrogram(wav_file):
    try:
        y, sr = librosa.load(wav_file, sr=16000)
        if len(y) < 16000: y = np.pad(y, (0, 16000 - len(y)))
        else: y = y[:16000]
        spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
        spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)
        return spectrogram_db
    except Exception as e:
        print(f"Erro processando {wav_file}: {e}")
        return None

spectrograms = []
labels = []
for i, keyword in enumerate(KEYWORDS):
    path = os.path.join(DATASET_PATH, keyword)
    print(f"Processando áudios em: {path}")
    if not os.path.exists(path):
        print(f"AVISO: Pasta não encontrada: {path}")
        continue
    for filename in os.listdir(path):
        if filename.endswith('.wav'):
             filepath = os.path.join(path, filename)
             spec = get_spectrogram(filepath)
             if spec is not None:
                 spectrograms.append(spec)
                 labels.append(i)

if not spectrograms:
     print("ERRO: Nenhum espectrograma foi gerado. Verifique seus arquivos .wav e caminhos.")
else:
    X = np.array(spectrograms)
    y = np.array(labels)
    X = X[..., np.newaxis]
    print(f"\n--- Processamento concluído. Formato dos dados: {X.shape} ---")


--- Processando áudios para espectrogramas ---
Processando áudios em: dataset/data/ligar
Processando áudios em: dataset/data/desligar
Processando áudios em: dataset/data/noise

--- Processamento concluído. Formato dos dados: (90, 128, 32, 1) ---


In [5]:
# Divisão de Dados e Treinamento do Modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

input_shape = X_train.shape[1:]
model = models.Sequential([
    layers.Input(shape=input_shape),
    layers.Resizing(32, 32),
    layers.Conv2D(8, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(16, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(len(KEYWORDS), activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

print("\n--- Iniciando o treinamento do modelo ---")
# Aumentei um pouco as épocas, mas monitore o gráfico para não overfitar
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=32)
print("--- Treinamento concluído! ---")

loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nAcurácia final no teste: {accuracy * 100:.2f}%")


--- Iniciando o treinamento do modelo ---
Epoch 1/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 354ms/step - accuracy: 0.3837 - loss: 11.3912 - val_accuracy: 0.3333 - val_loss: 8.4415
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - accuracy: 0.3832 - loss: 7.8846 - val_accuracy: 0.2778 - val_loss: 7.8232
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - accuracy: 0.3967 - loss: 5.6858 - val_accuracy: 0.3889 - val_loss: 2.1413
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 217ms/step - accuracy: 0.3641 - loss: 2.3157 - val_accuracy: 0.2778 - val_loss: 3.1766
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 186ms/step - accuracy: 0.3689 - loss: 3.1303 - val_accuracy: 0.2778 - val_loss: 2.9483
Epoch 6/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 316ms/step - accuracy: 0.4201 - loss: 2.1422 - val_accuracy: 0.5000 - val_loss: 1.4315
Epoch 7/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 161ms/step - accuracy: 0.5582 - loss: 1.1510 - val_accuracy: 0.3889 - val_loss: 2.7178
Epoch 8/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 218ms/step - accuracy: 0.4262 - los

In [7]:
# Conversão para TFLite (Float32 - SEM Quantização)
print("\n--- Convertendo o modelo para o formato TFLite (Float32) ---")

# Salva o modelo primeiro
model_save_path = 'keyword_model_saved'
model.export(model_save_path)

# Conversão simples, sem otimizações de quantização
converter = tf.lite.TFLiteConverter.from_saved_model(model_save_path)
tflite_model = converter.convert()

with open('keyword_model_float.tflite', 'wb') as f:
    f.write(tflite_model)

print("✅ Modelo Float32 salvo como keyword_model_float.tflite")


--- Convertendo o modelo para o formato TFLite (Float32) ---
Saved artifact at 'keyword_model_saved'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 32, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  138615474599312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138615444070544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138615474592784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138615444077840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138615444068240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138615444074384: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ Modelo Float32 salvo como keyword_model_float.tflite


In [8]:
# Converter para um Array em C
print("\n--- Convertendo para um arquivo C header (.h) usando 'xxd' ---")
!echo 'const unsigned char g_model[] = {' > model.h
!xxd -i keyword_model_float.tflite >> model.h
!echo '};' >> model.h
!echo 'const unsigned int g_model_len = sizeof(g_model);' >> model.h # Adiciona o tamanho
print("\n✅ Arquivo model.h (Float32) criado com sucesso! Faça o download.")


--- Convertendo para um arquivo C header (.h) usando 'xxd' ---

✅ Arquivo model.h (Float32) criado com sucesso! Faça o download.
